# **Chapter 8. Resolving Hallucinations**

Solar sometimes produces **hallucinations**—instances where the language model generates factually incorrect, fabricated, or irrelevant information—undermining the reliability of AI outputs, especially when accuracy is essential. 

The best way to resolve hallucinations in LLMs is using the technique named Retrieval-Augmented Generation(RAG), which integrates knowledge sources to provide more accurate and contextually relevant responses. In this cookbook, we do not cover RAG, but we introduce methods to mitigate hallucinations in Solar at the prompt level. 

This chapter introduces two methods to reduce or prevent hallucinations. 

- [**Method 1. Distinguishing Between Facts and Opinions**](#method1)

- [**Method 2. I don’t Know for Unknown Information**](#method2)

**Set up**

In [ ]:
from openai import OpenAI

# Retrieve the UPSTAGE_API_KEY variable from the IPython store
%store -r UPSTAGE_API_KEY

try:
    if UPSTAGE_API_KEY:
        print("Success!")
except NameError as ne:
    print(f"Since, {ne}")
    print("Please, insert your API key.")
    UPSTAGE_API_KEY = input("UPSTAGE_API_KEY =")

# Set your API key: 
# UPSTAGE_API_KEY = " " ←- Insert your API key here. 

client = OpenAI(
    api_key= UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

config_model = {
    "model": "solar-pro",
    "max_tokens": 996,
    "temperature": 0.5,
    "top_p": 1.0,
}

def get_completion(messages, system_prompt="", config=config_model):
    try:
        if system_prompt:
            messages = [{"role": "system", "content": system_prompt}] + messages

        message = client.chat.completions.create(messages=messages, **config)
        return message.choices[0].message.content
    
    except Exception as e:
        print(f"Error during API call: {e}")
        return None

Success!


---

<a id="method1"></a>
## **8.1 Distinguishing Between Facts and Opinions**

Here is a sentence where Solar produces a hallucination.

In [ ]:
# Imitative Falsehood
message = [
    {
        "role": "user",
        "content": "What is the original distance of a marathon?"
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

The original distance of a marathon is 26.2 miles, or 42.195 kilometers. This distance was standardized in 1921 and is based on the legendary run of a Greek soldier named Pheidippides, who is said to have run from the battlefield of Marathon to Athens to deliver news of a victory. 




Although the standard distance of a marathon is 42.195 kilometers (26.22 miles), it is based on the route from the start at Windsor Castle to the royal entrance of the White City Stadium. The original distance of the marathon was approximately 40 kilometers (24.85 miles, or around 25 miles). The model’s response reflects a common misconception present in its training data.

When large language models (LLMs) are trained on inaccurate data, they may unintentionally reinforce these errors, resulting in factually incorrect outputs, often referred to as '**imitative falsehoods**' ([Lin et al., 2022](https://arxiv.org/abs/2109.07958)).

To reduce hallucinations in response to the question, "*What is the original distance of a marathon?*", the prompt can be structured to separate `'fact' and 'your opinion'` in the answer.

In [ ]:
message = [
    {
        "role": "user",
        "content": """What is the original distance of a marathon?

Make a clear distinction between the fact and your opinion. 

-The Fact: 
-Your Opinion: """
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

-The Fact: The original distance of a marathon is approximately 25 miles or 42 kilometers. This distance is based on the legend of Pheidippides, a Greek messenger who ran from the city of Marathon to Athens to announce the Greeks' victory over the Persians in the Battle of Marathon in 490 BC.

-Your Opinion: In my opinion, the original distance of a marathon is a significant and iconic distance in the world of running and athletics. It represents a great challenge and accomplishment for runners, and the history and tradition behind it add to its allure and prestige. However, it's important to note that the actual distance may have varied over time and may not have been precisely measured. 




Now, we will tackle a more complicated task. We will provide an article to Solar and ask questions about it. 

Here is a question: What are the environmental impacts mentioned in the article regarding the early release of seasonal products like the PSL?

**Article’s Source**: [The First Falling Leaf and a Psychologist’s Explanation for the Pumpkin Spice Latte Craze](https://www.inc.com/nick-hobson/the-first-falling-leaf-a-psychologists-explanation-for-pumpkin-spice-latte-craze.html)

In [ ]:
article = """The First Falling Leaf and a Psychologist's Explanation for the Pumpkin Spice Latte Craze
It signals a fresh start. And it's Starbucks' biggest success story.

As the first leaf falls, signaling the subtle shift from summer to autumn, something stirs within us. It's not just the weather that changes; it's our entire psychological and emotional landscape. This transition has spawned a cultural phenomenon: the Pumpkin Spice Latte(PSL) craze. Today, on August 22nd, Starbucks released the PSL earlier than ever before. It's a day earlier than last year, but the sentiment is the same: It signals a natural shift in our consumer preferences.

The psychological impact of seasonal changes
The end of August and the beginning of September marks a significant psychological shift. Summer holidays wind down, children go back to school, and workplaces ramp up their efforts for the final quarter of the year. These changes are more than just logistical; they deeply influence our emotions, behavior, and interactions. Psychologically, this period represents a fresh stat-a concept well-documented in research. Known as the “fresh start effect,” this phenomenon suggests that people are more motivated to pursue goals and adopt new behaviors at the start of a new period, whether it's a week, a month, or a season.
The fresh start effect could be seen as a powerful driver behind the PSL craze. As the days grow shorter and the air cooler, we instinctively seek comfort and familiarity. The PSL, with its warm spices and association with cozy autumn days, provides that comfort. It's a ritual that signals the beginning of a new season, much like the first falling leaf.

The role of marketers in harnessing natural periods of change
Marketers and brand strategists have long understood the power of seasonal changes to influence consumer behavior. The PSL is a prime example of how a product can become synonymous with a season, creating a sense of anticipation and urgency that drives sales. But there's a delicate balance to be struck. As Starbucks has released the PSL earlier and earlier each year, there's a risk that the product could lose its association with that "Augtober" timing, diluting its appeal. The earlier release may satisfy immediate demand, but it also raises the question: Are we pushing too hard against the natural rhythms that make seasonal products so special? My own research on rituals has shown that the best rituals are those that aren't messed with or changed. So, perhaps, Starbucks should pick a date at the end of August, the beginning of September, and stick with it.
Successful marketers understand that the timing of a product launch is as crucial as the product itself. By aligning product releases with natural or culturally significant transitions, they can tap into the fresh start effect, creating a sense of newness and relevance. However, pushing too far ahead of these natural changes can backfire, leading to consumer fatigue and diminishing returns. The PSL's success lies in its ability to capture the essence of autumn, and if that connection is lost, the product may lose its allure.

As the first leaf falls this year, and the scent of pumpkin spice wafts through the air earlier than ever, it's worth considering the psychological and emotional forces at play. The PSL is more than just a drink. It's a signal of change, a comforting ritual, and a testament to the power of seasons to shape our lives. Marketers and business leaders, take note: In a world that's constantly changing, celebrating the moments of transition is not just good business strategy, it's human culture at its finest."""

In [ ]:
message = [
    {
        "role": "user",
        "content": """Q: What are the environmental impacts mentioned in the article regarding the early release of seasonal products like the PSL? 
-- 
{article}
""".format(article=article)
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

The article mentions two environmental impacts related to the early release of seasonal products like the PSL. First, it discusses the potential risk of diluting the product's appeal if it is released too early, which could lead to consumer fatigue and diminishing returns. Second, the article suggests that pushing too hard against natural rhythms can backfire, implying that maintaining a balance between product launches and natural transitions is crucial for long-term success. 




The provided article does not mention environmental impacts related to the early release of seasonal products like the PSL. Instead, it focuses on the psychological effects of seasonal transitions on consumer behavior and marketing strategies.

This issue can be resolved by requesting Solar to provide **evidence first and then answer**. The prompt should extract the most relevant and accurate sentences from the given text.

In the previous example, we used a prompt to distinguish between 'facts' and 'your opinion,' but this time, we used a similar prompt to distinguish between `'the evidence you found' and 'your final answer.'` By doing this, when Solar is instructed to find evidence, it can locate information, which helps reduce the rate of hallucinations in longer texts.

In [ ]:
message = [
    {
        "role": "user",
        "content": """What are the environmental impacts mentioned in the article regarding the early release of seasonal products like the PSL?  
Find the most accurate and relevant sentence in the given text. 
Observe the response format:

- Evidence You Found:  
- Your Final Answer:  

--
<Text>
{article}
</Text>
""".format(article=article)
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

No environmental impacts are mentioned in the article regarding the early release of seasonal products like the PSL. 




- Remember what we learned in Chapter 6: for longer documents or texts, it’s better to use a structured prompt.

---

<a id="method2"></a>
## **8.2 `Use References; Say "I don't know"`**

To prevent hallucinations, ensure Solar responds with "`I don't know`" for unknown information when asked about current or up-to-date details after its knowledge cut-off. Alternatively, provide the current date and year before prompting an answer or supply Solar with the relevant information you wish to inquire about to help mitigate hallucinations. 

Here’s another example: Solar may produce hallucinations when it lacks data on recent events.

In [ ]:
message = [
    {
        "role": "user",
        "content": "Which city hosted the most recent summer Olympics?"
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

The most recent summer Olympics were held in Tokyo, Japan in 2021. 




We believed that the most recent Summer Olympics took place in Paris, France, in 2024. This occurs because LLMs retain outdated information, illustrating a time-shift phenomenon in their responses. 


In [ ]:
# This problem can be solved as followed:

message = [
    {
        "role": "user",
        "content": """Which city hosted the most recent summer Olympics? It is now the year 2024.
        If you don't have updated information, please ONLY respond with 'I don't know.' """ 
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

I don't know. 




In [ ]:
# Let us try another example.

message = [
    {
        "role": "user",
        "content": "Who most recently won the Nobel Prize in Literature?"
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

The most recent winner of the Nobel Prize in Literature is Annie Ernaux. She was announced as the recipient in 2023 for her work "autobiographical narrative that gives the inherent worth of the human experience its rightful place." 




We knew the fact that the 2024 Nobel Prize in Literature was awarded to South Korean author Han Kang for her intense poetic prose that confronts historical traumas and exposes the fragility of human life.

We can resolve this issue using two methods. One is to use an *‘I don’t know’* prompt and, the other is to add additional information to the prompt.

**Method #1. I don't Know**

In [ ]:
message = [
    {
        "role": "user",
        "content": """Who most recently won the Nobel Prize in Literature in 2024? 
It is now the year 2024. If you don't have updated information, please ONLY respond with 'I don't know.'"""
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

I don't know. 




**Method #2. Additional Relevant Information**

In [ ]:
message = [
    {
        "role": "user",
        "content": """Who most recently won the Nobel Prize in Literature?  
Respond based on the following information: 
Han Kang is the first South Korean writer and the first Asian woman to receive the Nobel Prize in Literature in 2024. 
Her works are celebrated for their poetic and experimental style, addressing complex themes such as trauma, violence, and the human condition. 
This recognition highlights the growing global appreciation for Korean literature and culture, further elevating South Korea's presence in the international literary community."""
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

The most recent winner of the Nobel Prize in Literature is Han Kang. 




---

## **8.3 Practice**

**Exercise #1: Create a prompt that prevents hallucinations for the following questions**

Who was elected president in the 2024 U.S. presidential election?  

In [ ]:
message = [
    {
        "role": "user",
        "content": """ """ # ←- Insert your prompt here.
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

**Exercise #2: Read the following Starbucks PSL article and write a prompt that answers the following question accurately**

Question: How has the earlier release of the PSL affected consumer behavior, according to detailed statistical data? 

In [ ]:
prompt = """  Text: {article}""" # ←- Insert your prompt here.

message = [
    {
        "role": "user",
        "content": prompt.format(article=article) 
    }
]

response = get_completion(messages=message)
print(response, "\n\n")

- Answer: The text does not provide numerical data on how the earlier release of the PSL has affected consumer behavior.

One of the best methods for resolving hallucinations is RAG (Retrieval-Augmented Generation), which grounds the LLMs during generation by conditioning on relevant documents retrieved from an external knowledge source. As this cookbook is highly focused on prompt engineering only, we will address some prompt engineering lessons in later chapters

**Reference**

Huang, L., Yu, W., Ma, W., Zhong, W., Feng, Z., Wang, H., ... & Liu, T. (2023). *A survey on hallucination in large language models: Principles, taxonomy, challenges, and open questions.* arXiv preprint arXiv:2311.05232.